In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext, StorageContext, load_index_from_storage
import os
from langchain import HuggingFaceHub
import sys
import logging

sys.path.append('../')
from apikeys import huggingface_key


In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key

In [4]:
llm = LLMPredictor(llm=HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct"))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

In [5]:
if not os.path.exists("./storage"):
    # load the documents and create the index
    documents = SimpleDirectoryReader("../data").load_data()
    index = VectorStoreIndex.from_documents(documents,service_context=service_context)
    # store it for later
    index.storage_context.persist()
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context, service_context=service_context)


INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


In [6]:
query_engine = index.as_query_engine()
response = query_engine.query("How often should I feed my newborn baby?")
print(response)


According to the American Academy of Pediatrics, breast-fed babies should be fed at least 8 to 12 times a day. This may be more frequent than every 2 hours. It is recommended to feed your baby whenever he shows signs of hunger, which may be more frequent than every 2 hours.
